In [10]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import glob
import os

import matplotlib.pyplot as plt

in this notebook varya is trying to calculate the climatic (temperature) parameters for the centroid (wgs) coordinates of the catchments 

#### to do here (for the "last" 30 years 1990-2021): 

- read the table with coordinates 
- select only needed points in each file (loop through) 
- calculate the thing (loop? - when needed i guess) 

In [ ]:
# # temperature parameters of interest: 

# 1. Mean annual 
# 2. Jan temp mean 
# 3. July temp mean 
# 4. How many times over 0 - frost thaw cycles(?)  
# 5. Mean monsoon (no monsoon?)  
# 6. Monthly average -> cv (std/average) 
# 7. Permafrost -> part of the year that is lower then -2 (or smth) - or some measure to convert air temp to surface temp 
# 8. (Or part of the year when its close) 




# # 1. get coordinates:
# '''(NB: be carefil with x and y) 
# x - 85, y - 28
# longitude - 85, latitude - 28 ''' 

# lats = catchments.y_wgs
# lons = catchments.x_wgs


### 0. catchments and coordinates

In [2]:
table = pd.read_csv('morphometrical_params.csv', index_col = 0)
table_coords = table[['x_wgs', 'y_wgs']]

In [3]:
# table

In [4]:
table_coords.head()#.columns

,x_wgs,y_wgs
0,85.622,28.207
1,85.617,28.205
2,85.591,28.205
3,85.603,28.226
4,85.591,28.184


#### deal with .nc files first: create a file with subset of points for each year. deal with them after

In [ ]:
# %%time

# # loop through:
# # 1. select: 

# # check coords: what is x what is y

# # catch: x - 26 y - 28 
# # (x = median_t2m_regions.longitude, y = median_t2m_regions.latitude,

# # open one .nc to see whats going on 

# lats = table_coords.y_wgs
# lons = table_coords.x_wgs

# testfile = xr.open_dataset('/Users/varyabazilova/Desktop/era5land/temperature2m/temp_2m_year2021.nc')
# ds_sel = testfile.sel(longitude=lons, latitude=lats, method='nearest')#.mean(dim = 'time')
# ds_sel.to_netcdf('/Users/varyabazilova/Desktop/era5land/temperature2m/temp2m_2021_points.nc')
# # ds_sel.mean(dim = 'time')


#### read new point files 

In [5]:
%%time
points = xr.open_mfdataset('/Users/varyabazilova/Desktop/era5land/points_subset_t2m/*.nc')

CPU times: user 266 ms, sys: 1.08 s, total: 1.35 s
Wall time: 12 s


#### mean anual temperature

In [6]:
# just mean of all data:

mean = points.mean(dim='time')
# 

In [7]:
meant2m = mean.t2m

In [9]:
meant2m.shape

(742, 742)

In [ ]:
%%time

path = '/Users/varyabazilova/Desktop/era5land/points_subset_t2m'

# compute mean for each year and then means among them all 

# mean_annual = xr.Dataset()
means = pd.DataFrame()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # select the coordinates: 
    dsmean = ds.mean(dim = 'time')
    # to df:
    df = dsmean.to_dataframe()#.drop_duplicates()
    # append:
    means = means.append(df)
    print(df)

In [ ]:
%%time
# read all files - years: 1990-2021



path = '/Users/varyabazilova/Desktop/era5land/temperature2m/'


subset = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # select the coordinates: 
    ds_sel = ds.sel(longitude=lons, latitude=lats, method='nearest')
    # merge all back together 
    subset = xr.merge([subset, ds_sel])


In [ ]:
# mean -> to scv: 
subset = subset.mean(dim = 'time') 
df = subset.to_dataframe()
df.to_csv('/Users/varyabazilova/Desktop/era5land/temperature2m/annual_mean.csv')

In [ ]:
# file = xr.open_dataset('/Users/varyabazilova/Desktop/era5land/temperature2m/temperature_2m_1960.nc', decode_coords="all")

In [ ]:
# file 

In [ ]:
# select things 

# calculate things 

# iterate to save to csv (see other notebook where you did that) 